In [1]:
!pip install torch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import PIL

from torch.autograd import Variable

# check the system we're on, gpu
use_cuda = torch.cuda.is_available()

np.load.__defaults__=(None, True, True, 'ASCII')

In [2]:
class HyperParameters():
    def __init__(self):
        self.data = '../Datasets/sketchrnn_chair.npz'
        self.encoder_layer_size = 64
        self.decoder_layer_sze = 128
        # latent vector
        self.Nz = 32
        
        #return
        self.M = 20
        self.dropout = 0.9
        self.batch_size = 100
        self.eta_min = 0.01
        self.R = 0.99995
        self.KL_min = 0.2
        self.wKL = 0.5
        self.lr = 0.001
        self.lr_decay = 0.9999
        self.min_lr = 0.00001
        self.grad_clip = 1.
        self.temperature = 0.4
        self.max_seq_length = 200

hp = HyperParameters()

In [3]:
"""
Padding - each stroke data has different lengths of vectors
- take the largest vector and make all of them the same
"""


'\nPadding - each stroke data has different lengths of vectors\n- take the largest vector and make all of them the same\n'

In [4]:
class DataLoader():
    def __init__(self, file):
        self.file = file
        
    def load_npz(self):
        data = np.load(file=self.file, encoding='latin1', allow_pickle=True)
        return data

dataLoader = DataLoader(hp.data)
data = dataLoader.load_npz()

In [26]:
class DataHandler():
    def __init__(self, data):
        self.data = data
   
    def padding_three_to_five(self):
        max_len = 0
        for datatype in self.data:
            for strokedata in self.data[datatype]:
                if strokedata.shape[0] > max_len:
                    max_len = strokedata.shape[0]
        for datatype in self.data:
            for strokedata in self.data[datatype]:
                len = strokedata.shape[0]
                pen_down = np.empty(len)
                pen_stop = np.zeros((len, 1))
                pen_stop[len - 1] = 1                   
                pen_down[:] = 1 - strokedata[: , 2]     # 0 to 1, 1 to 0
                strokedata = np.insert(strokedata, 2, pen_down, 1)
                strokedata = np.append(strokedata, pen_stop, 1)
                # do padding till max_len
                if len < max_len:
                    padding_arr = np.zeros((max_len - len, 5))
                    padding_arr[:, 4] = 1
                    strokedata = np.append(strokedata, padding_arr, 0)                                           
                print(strokedata)
                    
                    

datahandler = DataHandler(data)
datahandler.padding_three_to_five()

(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


...

(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)
(66, 5)


In [23]:
arr = np.asarray([1, 2, 3])
np.insert(arr, 1, 0)
print(arr)

[1 2 3]
